##### Install modules

In [ ]:
!pip install praw
!pip install -U sentence-transformers
!pip install annoy
!pip install langdetect

     |████████████████████████████████| 176 kB 7.6 MB/s 
     |████████████████████████████████| 52 kB 884 kB/s 
     |████████████████████████████████| 78 kB 3.9 MB/s 
     |████████████████████████████████| 3.1 MB 28.5 MB/s 
     |████████████████████████████████| 3.3 MB 70.5 MB/s 
     |████████████████████████████████| 1.2 MB 57.2 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 895 kB 76.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=49ad4cae998e7594284c4714203e42517966771ac20b70f2c57e73168aa9020e
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |█

##### Import dependencies

In [ ]:
import praw
from requests import Session
import urllib3
import nltk
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
import spacy
import numpy as np
from statistics import mean
from sklearn.metrics.pairwise import cosine_similarity
import torch
import random
from langdetect import detect

##### Useful functions

In [ ]:
def exit_condition(text):
  exit_condition = False
  listOfWords = ['thanks', 'thank you', 'no problem', 'bye', 'good day', 'c u', 'ok cool']
  if any(word in text.lower() for word in listOfWords):
    exit_condition = True

  return exit_condition

def ANN(test_sentence,saved_ann):
  model_sent = SentenceTransformer('all-MiniLM-L6-v2')
  sentence_embeddings = model_sent.encode(test_sentence)
  f = sentence_embeddings.shape[0]
  u = AnnoyIndex(f, 'angular')
  u.load(f'{saved_ann}') # super fast, will just mmap the file
  closest, dist = u.get_nns_by_vector(sentence_embeddings, 1000, include_distances=True) # will find the 1000 nearest neighbors
  return closest[:10], dist[:10]

##### Set up reddit instance

In [ ]:
### Set up reddit instance
reddit = praw.Reddit(
    client_id='wAuzk5-Fus41St96ccW7sw',
    client_secret="FrWDWIXJpptTmeeMkusMj6e9ra4gNg",
    user_agent="web_scraper_2000"
)


#### Function for Scraper, ANN maker and releent sentence maker

In [ ]:
def releventor(subreddit_choice):
  ### Get list of submission titles
  model_sent = SentenceTransformer('all-MiniLM-L6-v2')
  for topic in subreddit_choice:
      title_list = []
      for submission in reddit.subreddit(topic).hot(limit=None):
          title_list.append(submission.title)

      ### Set up annoy model for scraped titles
      sentence_embeddings = model_sent.encode(title_list)
      f = len(sentence_embeddings[0])
      t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
      for i in range(len(sentence_embeddings)):
          v = sentence_embeddings[i]
          t.add_item(i, v)

      t.build(10)  # 10 trees
      t.save('titles.ann')

      #### Scrape the comments
      comment_list = []
      for submission in reddit.subreddit(topic).top(limit=100):
          try:
              submission.comments.replace_more(limit=5)
              for comment in submission.comments.list():
                  comment_list.append(comment.body)
          except:
              pass

      ### Refine the comments
      selected_sent = []
      for line in comment_list:
        if 'https' in line or 'www' in line:
          continue
        try:
          if detect(line) != 'en':
            continue
        except:
          pass
        sent, dist = ANN(line, 'titles.ann')
        if mean(dist) < 1 and len(line.strip(",'\n ")) > 2:
            selected_sent.append(line.strip('\n'))
            if len(selected_sent) == 3000:
                break
        print(len(comment_list), "vs", len(selected_sent))

      ## Create refined text document
      with open(f'{topic}_relevent.txt', 'w', encoding='utf-8') as f:
          for item in selected_sent:
              f.write("%s\n" % item)


      #### Create 
      sentence_embeddings = model_sent.encode(selected_sent)
      f = len(sentence_embeddings[0])
      t = AnnoyIndex(f, 'angular')
      for i in range(len(sentence_embeddings)):
          v = sentence_embeddings[i]
          t.add_item(i, v)

      t.build(10) # 10 trees
      t.save(f'{topic}.ann')

In [ ]:
releventor(['dogs'])

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt

29905 vs 0
29905 vs 0
29905 vs 0
29905 vs 0
29905 vs 0
29905 vs 0
29905 vs 0
29905 vs 0
29905 vs 0
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 1
29905 vs 2
29905 vs 2
29905 vs 2
29905 vs 2
29905 vs 2
29905 vs 2
29905 vs 2
29905 vs 2
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 3
29905 vs 4
29905 vs 4
29905 vs 4
29905 vs 4